In [3]:
import sys
import os

custom_path = "../../../src"
if custom_path not in sys.path:
    sys.path.insert(0, custom_path)

In [3]:
!pip install torch

  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
    --------------------------------------- 3.7/216.1 MB 19.8 MB/s eta 0:00:11
   - -------------------------------------- 7.9/216.1 MB 20.3 MB/s eta 0:00:11
   -- ------------------------------------- 12.3/216.1 MB 19.8 MB/s eta 0:00:11
   --- ------------------------------------ 16.8/216.1 MB 19.9 MB/s eta 0:00:11
   --- ------------------------------------ 21.0/216.1 MB 20.1 MB/s eta 0:00:10
   ---- ----------------------------------- 25.4/216.1 MB 20.1 MB/s eta 0:00:10
   ----- ---------------------------------- 30.1/216.1 MB 20.1 MB/s eta 0:00:10
   ------ --------------------------------- 34.6/216.1 MB 20.3 MB/s eta 0:00:09
   ------- -------------------------------- 39.1/216.1 MB 20.3 MB/s eta 0:00:09
   -------- ------------------------------- 43.3/216.1 MB 20.2 MB/s eta 0:00:09
   ---

In [4]:
import datetime
import pandas as pd
from socio4health import Extractor
from socio4health.harmonizer import Harmonizer
from socio4health.utils import harmonizer_utils

In [5]:
col_online_extractor = Extractor(input_path="https://microdatos.dane.gov.co/index.php/catalog/643/get-microdata",
                                 down_ext=['.CSV','.csv','.zip'],
                                 sep=';',
                                 output_path="../CNPV2018",
                                 key_words=["73_Tolima.zip"],
                                 depth=0)
col_CNPV = col_online_extractor.extract()

2025-07-25 09:49:48,143 - INFO - ----------------------
2025-07-25 09:49:48,145 - INFO - Starting data extraction...
2025-07-25 09:49:48,147 - INFO - Extracting data in online mode...
2025-07-25 09:49:48,150 - INFO - Scraping URL: https://microdatos.dane.gov.co/index.php/catalog/643/get-microdata with depth 0
2025-07-25 09:49:49,808 - INFO - Successfully saved links to Output_scrap.json.
2025-07-25 09:49:49,839 - INFO - Downloading files to: ../CNPV2018
2025-07-25 09:50:09,075 - INFO - Extracted file: ../CNPV2018\CNPV2018_1VIV_A2_73_20250725095009.CSV
2025-07-25 09:50:09,081 - INFO - Extracted file: ../CNPV2018\CNPV2018_2HOG_A2_73_20250725095009.CSV
2025-07-25 09:50:09,085 - INFO - Extracted file: ../CNPV2018\CNPV2018_3FALL_A2_73_20250725095009.CSV
2025-07-25 09:50:09,089 - INFO - Extracted file: ../CNPV2018\CNPV2018_5PER_A2_73_20250725095009.CSV
2025-07-25 09:50:09,094 - INFO - Extracted file: ../CNPV2018\CNPV2018_MGN_A2_73_20250725095009.CSV
Processing files:   0%|          | 0/5 [00

In [6]:
import dask.dataframe as dd
import pandas as pd
from functools import reduce

# Paso 1: Convertir todos a dask.DataFrame si no lo son ya
lista_dfs_normalizados = []
for df in col_CNPV:
    if isinstance(df, pd.DataFrame):
        ddf = dd.from_pandas(df, npartitions=1)
    elif hasattr(df, 'compute'):  # ya es Dask DataFrame
        ddf = df
    else:
        raise ValueError("Tipo no soportado:", type(df))
    lista_dfs_normalizados.append(ddf)

# Paso 2: Obtener columnas comunes a todos los dataframes
keys = sorted(set(lista_dfs_normalizados[0].columns))
for ddf in lista_dfs_normalizados[1:]:
    keys = list(set(keys) & set(ddf.columns))  # intersección de columnas

# Paso 2: Remover columnas duplicadas no clave antes de merge
def remove_duplicate_columns(df, keys):
    # Solo dejamos las columnas que no están duplicadas en la unión
    return df[[col for col in df.columns if col in keys or col not in keys]]

dfs_limpios = [df.drop_duplicates(subset=keys) for df in lista_dfs_normalizados]

# Paso 3: Hacer merge sucesivos
df_merged = reduce(
    lambda left, right: dd.merge(
        left, right[[col for col in right.columns if col not in left.columns or col in keys]],
        on=keys, how='outer'
    ),
    dfs_limpios
)


In [7]:
prueba =[df_merged]

In [8]:
har = Harmonizer()
har.similarity_threshold = 0.9
har.nan_threshold = 1
# dfs = har.vertical_merge(col_CNPV)
dfs = har.drop_nan_columns(prueba)
available_columns = har.get_available_columns(dfs)

2025-07-25 09:50:18,406 - INFO - Dropping columns with majority NaN values...
2025-07-25 09:50:19,322 - DEBUG - open file: c:/Users/jmont/OneDrive - Universidad de los andes/Uniandes/2025/Harmonize/socio4health/docs/source/notebooks/../CNPV2018/CNPV2018_1VIV_A2_73_20250725095009.CSV
2025-07-25 09:50:19,328 - DEBUG - open file: c:/Users/jmont/OneDrive - Universidad de los andes/Uniandes/2025/Harmonize/socio4health/docs/source/notebooks/../CNPV2018/CNPV2018_3FALL_A2_73_20250725095009.CSV
2025-07-25 09:50:19,333 - DEBUG - open file: c:/Users/jmont/OneDrive - Universidad de los andes/Uniandes/2025/Harmonize/socio4health/docs/source/notebooks/../CNPV2018/CNPV2018_5PER_A2_73_20250725095009.CSV
2025-07-25 09:50:19,335 - DEBUG - open file: c:/Users/jmont/OneDrive - Universidad de los andes/Uniandes/2025/Harmonize/socio4health/docs/source/notebooks/../CNPV2018/CNPV2018_MGN_A2_73_20250725095009.CSV
2025-07-25 09:50:19,352 - DEBUG - open file: c:/Users/jmont/OneDrive - Universidad de los andes/Un

2025-07-25 10:25:35,254 - DEBUG - open file: c:/Users/jmont/OneDrive - Universidad de los andes/Uniandes/2025/Harmonize/socio4health/docs/source/notebooks/../CNPV2018/CNPV2018_3FALL_A2_73_20250725095009.CSV
2025-07-25 10:25:35,254 - DEBUG - open file: c:/Users/jmont/OneDrive - Universidad de los andes/Uniandes/2025/Harmonize/socio4health/docs/source/notebooks/../CNPV2018/CNPV2018_1VIV_A2_73_20250725095009.CSV
2025-07-25 10:25:35,255 - DEBUG - open file: c:/Users/jmont/OneDrive - Universidad de los andes/Uniandes/2025/Harmonize/socio4health/docs/source/notebooks/../CNPV2018/CNPV2018_5PER_A2_73_20250725095009.CSV
2025-07-25 10:25:35,255 - DEBUG - open file: c:/Users/jmont/OneDrive - Universidad de los andes/Uniandes/2025/Harmonize/socio4health/docs/source/notebooks/../CNPV2018/CNPV2018_MGN_A2_73_20250725095009.CSV
2025-07-25 10:25:35,255 - DEBUG - open file: c:/Users/jmont/OneDrive - Universidad de los andes/Uniandes/2025/Harmonize/socio4health/docs/source/notebooks/../CNPV2018/CNPV2018_

In [31]:
len(available_columns)

92

In [15]:
raw_dict = pd.read_excel('../../Socio4HealthData/Dictionaries/Colombia/Diccionario Crudo CNVP.xlsx')
dic = harmonizer_utils.standardize_dict(raw_dict)
dic = harmonizer_utils.translate_column(dic, "question", language="en")
dic = harmonizer_utils.translate_column(dic, "description", language="en")
dic = harmonizer_utils.translate_column(dic, "possible_answers", language="en")
dic = harmonizer_utils.classify_rows(dic, "question_en", "description_en", "possible_answers_en",
                                        new_column_name="category",
                                        MODEL_PATH="../../Socio4HealthData/input/bert_finetuned_classifier")

2025-07-17 15:00:44,650 - WARNING - c:\Users\jmont\OneDrive - Universidad de los andes\Uniandes\2025\Harmonize\socio4health\tests\../src\socio4health\utils\harmonizer_utils.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  column.replace(r'^\s*$', np.nan, regex=True)

2025-07-17 15:00:45,066 - WARNING - c:\Users\jmont\OneDrive - Universidad de los andes\Uniandes\2025\Harmonize\socio4health\tests\../src\socio4health\utils\harmonizer_utils.py:78: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to sile

question translated
description translated
Rows with contents longer than 5000 characters are cut off
Rows with contents longer than 5000 characters are cut off
possible_answers translated


In [9]:
dic = pd.read_excel('../../../../Socio4HealthData/Dictionaries/Colombia/Diccionario Crudo CNVP Categorizado.xlsx')

In [10]:
dic

,Unnamed: 0,variable_name,question,description,value,size,possible_answers,question_en,description_en,possible_answers_en,category
0,0,p_enfermo,algun problema de salud en los últimos 30 días...,NaN,1; 2; 9,NaN,si; no; no informa,"some health problem in the last 30 days, witho...",NaN,Yeah; No; It does not inform,Business
1,1,condicion_fisica,alguna dificultad en su vida diaria,NaN,1; 2; 9,NaN,si; no; no informa,Some difficulty in your daily life,NaN,Yeah; No; It does not inform,Business
2,2,pa_asistencia,asistencia escolar (de forma presencial o virt...,NaN,1; 2; 9;,NaN,si; no; no informa; no aplica,School Asistance (in person the virtual way),NaN,Yeah; No; does not inform; It does not apply,Education
3,3,pa_lo_atendieron,atención del problema de salud,NaN,1; 2; 9;,NaN,si; no; no informa; no aplica,Health problem care,NaN,Yeah; No; does not inform; It does not apply,Nonstandard job
4,4,pa1_calidad_serv,calidad de la prestación del servicio de salud,NaN,1; 2; 3; 4; 9;,NaN,muy bueno; bueno; malo; muy malo; no informa; ...,quality of health service,NaN,very good; Well; bad; very bad; does not infor...,Business
...,...,...,...,...,...,...,...,...,...,...,...
167,167,p_quehizo_ppal,tratamiento principal del problema de salud,NaN,1; 2; 3; 4; 5; 6; 7; 8; 9; 99;,NaN,acudió a la entidad de seguridad social en sal...,Main treatment of the health problem,NaN,He went to the Social Security entity in healt...,Education
168,168,uva_uso_unidad,uso de la unidad,NaN,1; 2; 3; 4,NaN,vivienda; mixto (espacio independiente y separ...,Use of Unit,NaN,dwelling; mixed (independent and separate spac...,Housing
169,169,pa21_cod_vitsa,vitsa de pertenencia,NaN,1; 2; 3; 4; 5; 6; 7; 8; 9;,NaN,1_bolochok; 2_churon; 3_mijay; 4_ghuso – ruso;...,VITSA of belonging,NaN,1_bolchok; 2_Churon; 3_Mijay; 4_Ghuso - Russia...,Identification
170,170,uva_esta_areaprot,vivienda en un área protegida,NaN,1; 2; 9;,NaN,si; no; no informa; no aplica,Housing in a protected area,NaN,Yeah; No; does not inform; It does not apply,Housing


In [11]:
dic[dic['category'] == "Business"]

,Unnamed: 0,variable_name,question,description,value,size,possible_answers,question_en,description_en,possible_answers_en,category
0,0,p_enfermo,algun problema de salud en los últimos 30 días...,NaN,1; 2; 9,NaN,si; no; no informa,"some health problem in the last 30 days, witho...",NaN,Yeah; No; It does not inform,Business
1,1,condicion_fisica,alguna dificultad en su vida diaria,NaN,1; 2; 9,NaN,si; no; no informa,Some difficulty in your daily life,NaN,Yeah; No; It does not inform,Business
4,4,pa1_calidad_serv,calidad de la prestación del servicio de salud,NaN,1; 2; 3; 4; 9;,NaN,muy bueno; bueno; malo; muy malo; no informa; ...,quality of health service,NaN,very good; Well; bad; very bad; does not infor...,Business
5,5,fa4_cert_defun,certificado médico de defunción,NaN,1; 2; 3; 9,NaN,si; no; no sabe; no informa,Death Medical Certificate,NaN,Yeah; No; He doesn't know; It does not inform,Business
7,7,ua_clase,clase,NaN,1; 2; 3; 4,NaN,1_cabecera municipal; 2_centro poblado; 3_rura...,class,NaN,1_ municipal box; 2_ Populated Center; 3_ Disp...,Business
9,9,vb_acu,cuenta con servicio de acueducto,NaN,1; 2,NaN,si; no,has aqueduct service,NaN,Yeah; No,Business
10,10,vc_alc,cuenta con servicio de alcantarillado,NaN,1; 2,NaN,si; no,has sewerage service,NaN,Yeah; No,Business
11,11,va_ee,cuenta con servicio de energía eléctrica,NaN,1; 2,NaN,si; no,has electric power service,NaN,Yeah; No,Business
12,12,vd_gas,cuenta con servicio de gas natural conectado a...,NaN,1; 2; 9,NaN,si; no; no informa,has a natural gas service connected to a publi...,NaN,Yeah; No; It does not inform,Business
14,14,ve_recbas,cuenta con servicio de recolección de basura,NaN,1; 2; 9,NaN,si; no; no informa,It has garbage collection service,NaN,Yeah; No; It does not inform,Business


In [12]:
har.dict_df = dic
har.categories = ["Business"]
har.key_col = 'U_MPIO'
har.key_val = ['001']
filtered_ddfs = har.data_selector(prueba)

2025-07-25 10:25:44,257 - DEBUG - Filtering DataFrame for columns: ['P_ENFERMO', 'CONDICION_FISICA', 'PA1_CALIDAD_SERV', 'FA4_CERT_DEFUN', 'UA_CLASE', 'VB_ACU', 'VC_ALC', 'VA_EE', 'VD_GAS', 'VE_RECBAS', 'VE1_QSEM', 'FA3_EDAD_FALL', 'PA_HABLA_LENG', 'PA_HNVS', 'V_MAT_PISO', 'V_MAT_PARED', 'PA2_ANO_UHNV', 'U_SECC_RUR', 'U_SECT_RUR', 'FA1_NRO_FALL', 'P_TRABAJO', 'HA_NRO_FALL']
2025-07-25 10:25:44,257 - DEBUG - Available columns: ['TIPO_REG', 'U_DPTO', 'U_MPIO', 'UA_CLASE', 'U_EDIFICA', 'COD_ENCUESTAS', 'U_VIVIENDA', 'UVA_ESTATER', 'UVA1_TIPOTER', 'UVA2_CODTER', 'UVA_ESTA_AREAPROT', 'UVA1_COD_AREAPROT', 'UVA_USO_UNIDAD', 'V_TIPO_VIV', 'V_CON_OCUP', 'V_TOT_HOG', 'V_MAT_PARED', 'V_MAT_PISO', 'VA_EE', 'VA1_ESTRATO', 'VB_ACU', 'VC_ALC', 'VD_GAS', 'VE_RECBAS', 'VE1_QSEM', 'VF_INTERNET', 'V_TIPO_SERSA', 'L_TIPO_INST', 'L_EXISTEHOG', 'L_TOT_PERL', 'F_NROHOG', 'FA1_NRO_FALL', 'FA2_SEXO_FALL', 'FA3_EDAD_FALL', 'FA4_CERT_DEFUN', 'P_NROHOG', 'P_NRO_PER', 'P_SEXO', 'P_EDADR', 'P_PARENTESCOR', 'PA1_GRP

In [13]:
har.compare_with_dict(prueba)

Matches with dict_df: 91 (98.91%)


,Unmatched ddfs variable,Unmatched dict_df variables
0,COD_DANE_ANM,REGFALL
1,None,REGHOG
2,None,REGPER
3,None,REGVIVI


In [12]:
filtered_ddfs[0].compute()

2025-07-17 14:58:08,383 - DEBUG - open file: c:/Users/jmont/OneDrive - Universidad de los andes/Uniandes/2025/Harmonize/socio4health/tests/../CNPV2018/CNPV2018_2HOG_A2_73_20250717144528.CSV


,U_MPIO,U_SECC_RUR,U_SECT_RUR,PA1_CALIDAD_SERV,P_TRABAJO,VE1_QSEM
0,001,00,000,<NA>,1,3
1,001,00,000,<NA>,6,3
2,001,00,000,<NA>,<NA>,3
3,001,00,000,<NA>,7,3
4,001,00,000,<NA>,7,3
...,...,...,...,...,...,...
514951,001,00,000,<NA>,1,3
514952,001,00,000,<NA>,7,3
514953,001,00,000,<NA>,<NA>,<NA>
514954,001,00,000,<NA>,1,3
